<div class="alert alert-info">
Привет, Юлия! Меня зовут Светлана Чих и я буду проверять твой проект. Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе.

<div class="alert alert-success">
<b>👍 Успех:</b> Зелёным цветом отмечены удачные и элегантные решения, на которые можно опираться в будущих проектах.
</div>
<div class="alert alert-warning">
<b>🤔 Рекомендация:</b> Жёлтым цветом выделено то, что в следующий раз можно сделать по-другому. Ты можешь учесть эти комментарии при выполнении будущих заданий или доработать проект сейчас (однако это не обязательно).
</div>
<div class="alert alert-danger">
<b>😔 Необходимо исправить:</b> Красным цветом выделены комментарии, без исправления которых, я не смогу принять проект :(
</div>
<div class="alert alert-info">
<b>👂 Совет:</b> Какие-то дополнительные материалы
</div>
Давай работать над проектом в диалоге: если ты что-то меняешь в проекте по моим рекомендациям — пиши об этом.
Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>🎓 Комментарий студента:</b> Например, вот так.</div>
Пожалуйста, не перемещай, не изменяй и не удаляй мои комментарии. Всё это поможет выполнить повторную проверку твоего проекта быстрее.
 </div>

# Банки — Анализ оттока клиентов
## Цель - выявить сегменты клиентов банка, наиболее склонных к оттоку.

<div class="alert alert-danger">
<b>😔 Необходимо исправить:</b> Должно быть описание задачи, цель проекта и план его выполнения
</div>

In [1]:
pip install -U imbalanced-learn


Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
import numpy as np

from scipy import stats as st
from xgboost import  XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn import set_config
from sklearn.utils import shuffle
from numpy.random import RandomState
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, make_scorer, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline as sk_Pipeline
from imblearn.pipeline import Pipeline 
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.3f}'.format)

[1. Загрузка данных и знакомство с ними](#data_download)  
[2. Предобработка данных](#data_preprocessing)  
[3. Подготовка данных](#data_preparation)  
[4. Моделирование](#modeling)  
[5. Сегментирование](#segmentation)    
[6. Проверка статистических гипотез](#statistics)  
[7. Выводы и предложения](#conclusion)

<a id='data_download'></a>
### 1. Загрузка данных и знакомство с ними

In [6]:
path = "https://drive.google.com/uc?export=download&id=1-U61mhTz_N1ARjy2XSAZ7IlQqGjeqP0F"
df = pd.read_csv(path)

In [7]:
df.head()

,USERID,score,city,gender,age,equity,balance,products,credit_card,last_activity,EST_SALARY,churn
0,183012,850.000,Рыбинск,Ж,25,1,59214.820,2,0,1,75719.140,1
1,146556,861.000,Рыбинск,Ж,37,5,850594.330,3,1,0,86621.770,0
2,120722,892.000,Рыбинск,Ж,30,0,NaN,1,1,1,107683.340,0
3,225363,866.000,Ярославль,Ж,51,5,1524746.260,2,0,1,174423.530,1
4,157978,730.000,Ярославль,М,34,5,174.000,1,1,0,67353.160,1


<div class="alert alert-success">
<b>👍 Успех:</b> Датасет загружен и просмотрен
</div>

<a id='data_preprocessing'></a>
### 2. Предобработка данных

In [8]:
# приведем столбцы к нижнему регистру:
df.columns = df.columns.str.lower()

In [9]:
# переименуем столбец userid 
df = df.rename(columns={'userid' : 'user_id'})

In [10]:
df.head()

,user_id,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
0,183012,850.000,Рыбинск,Ж,25,1,59214.820,2,0,1,75719.140,1
1,146556,861.000,Рыбинск,Ж,37,5,850594.330,3,1,0,86621.770,0
2,120722,892.000,Рыбинск,Ж,30,0,NaN,1,1,1,107683.340,0
3,225363,866.000,Ярославль,Ж,51,5,1524746.260,2,0,1,174423.530,1
4,157978,730.000,Ярославль,М,34,5,174.000,1,1,0,67353.160,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   user_id        10000 non-null  int64  
 1   score          10000 non-null  float64
 2   city           10000 non-null  object 
 3   gender         10000 non-null  object 
 4   age            10000 non-null  int64  
 5   equity         10000 non-null  int64  
 6   balance        7705 non-null   float64
 7   products       10000 non-null  int64  
 8   credit_card    10000 non-null  int64  
 9   last_activity  10000 non-null  int64  
 10  est_salary     10000 non-null  float64
 11  churn          10000 non-null  int64  
dtypes: float64(3), int64(7), object(2)
memory usage: 937.6+ KB


In [12]:
df.describe()

,user_id,score,age,equity,balance,products,credit_card,last_activity,est_salary,churn
count,10000.000,10000.000,10000.000,10000.000,7705.000,10000.000,10000.000,10000.000,10000.000,10000.000
mean,171814.713,848.699,42.837,2.628,827794.307,1.874,0.680,0.523,147866.886,0.182
std,33708.238,65.449,12.129,1.981,1980614.147,0.800,0.466,0.499,139388.511,0.386
min,94561.000,642.000,18.000,0.000,0.000,0.000,0.000,0.000,2546.300,0.000
25%,142810.250,802.000,34.000,0.000,295554.160,1.000,0.000,0.000,75251.900,0.000
50%,172728.000,853.000,40.000,3.000,524272.200,2.000,1.000,1.000,119658.105,0.000
75%,201261.750,900.000,51.000,4.000,980705.850,2.000,1.000,1.000,174500.542,0.000
max,229145.000,1000.000,86.000,9.000,119113552.010,5.000,1.000,1.000,1395064.450,1.000


In [13]:
# Проверим пропуски
df.isna().sum()

user_id             0
score               0
city                0
gender              0
age                 0
equity              0
balance          2295
products            0
credit_card         0
last_activity       0
est_salary          0
churn               0
dtype: int64

In [14]:
# проверим на дубликаты
df.duplicated().sum()

1

In [15]:
df[df.duplicated()]

,user_id,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
9457,141945,929.000,Ярославль,М,33,0,NaN,1,1,0,381868.890,0


In [16]:
#удалим дубликат
df = df.drop_duplicates()

#### столбец user_id

In [17]:
df.user_id.duplicated().sum()

72

In [18]:
ids = df['user_id']

In [19]:
df[ids.isin(ids[ids.duplicated()])].sort_values("user_id")

,user_id,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
7694,116540,887.000,Ярославль,Ж,38,0,NaN,1,0,1,119247.610,0
1893,116540,883.000,Ярославль,Ж,55,1,362756.490,3,0,1,175920.480,1
4866,117943,855.000,Рыбинск,Ж,32,6,1036832.930,4,1,1,107792.710,1
7542,117943,880.000,Ярославль,Ж,40,0,NaN,1,1,0,137718.930,0
5863,120258,908.000,Рыбинск,Ж,38,4,2213581.630,2,0,1,160327.770,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2597,226719,990.000,Ярославль,М,37,4,14648692.140,2,0,0,934412.610,1
8205,227795,840.000,Рыбинск,М,34,2,350768.030,1,1,0,102036.140,1
8497,227795,839.000,Рыбинск,М,34,2,326593.140,2,1,0,103314.920,0
6457,228075,839.000,Ярославль,М,39,5,507199.850,3,0,1,85195.800,0


Существуют дубликаты user_id, их очень мало - 72 строки, похоже, что имеет место ошибка сбора данных, но на качество данных для наших целей это не влияет: user_id как таковой нам не нужен при анализе, можем эту проблему игнорировать.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

#### столбец score

In [20]:
fig = px.histogram(df, x='score')
fig.show()

Распределние баллов нормально, в целом нормальное, с выделяющимися пиками на уровне 875-925 баллов

#### стоблец city

In [21]:
df.city.value_counts().sort_values(ascending=False)

Ярославль    5905
Рыбинск      2663
Ростов       1431
Name: city, dtype: int64

Всего в данных представлено 3 города: Ярославль, Рыбинск, Ростов, больше всего клиентов из Ярославля

#### столбец gender

In [22]:
df.gender.value_counts()

М    5007
Ж    4992
Name: gender, dtype: int64

Клиенты разделяются на 2 пола примерно поровну

In [23]:
# Для целей дальнейшей обработки, заменим М - 1 Ж 0:
df.gender = df.gender.map({'М':1, 'Ж':0})

In [24]:
df.head()

,user_id,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
0,183012,850.000,Рыбинск,0,25,1,59214.820,2,0,1,75719.140,1
1,146556,861.000,Рыбинск,0,37,5,850594.330,3,1,0,86621.770,0
2,120722,892.000,Рыбинск,0,30,0,NaN,1,1,1,107683.340,0
3,225363,866.000,Ярославль,0,51,5,1524746.260,2,0,1,174423.530,1
4,157978,730.000,Ярославль,1,34,5,174.000,1,1,0,67353.160,1


#### столбец age

In [25]:
fig = px.histogram(df, x='age', title='Распределение клиентов банка по возрасту')
fig.show()

<div class="alert alert-danger">
<b>😔 Необходимо исправить:</b> Визуализация не отображается во всем разделе
</div>

Среди клиентов прсуствуют люди от 18 до 86 лет, возрастная пирамида сдвинута влево, больше всего молодых трудоспособных людей (30-45 лет)

#### столбец equity

In [26]:
df.equity.value_counts()

0    2591
5    1918
4    1850
3    1543
2    1052
1     774
6     161
7      80
8      17
9      13
Name: equity, dtype: int64

In [27]:
fig = px.histogram(df, x='equity', title='Распределение клиентов по наличию объектов собственности')
fig.show()

Больше всего клиентов вообще не обладают никакой собственностью, второе и третье места в рейтинге у "обеспеченных" клиентов с 5 и 4 объектами недвижимости

#### столбец balance

In [28]:
(df.balance.mean(), df.balance.median())

(827794.3065100583, 524272.2)

In [29]:
df.balance.isna().sum()

2294

In [30]:
df[df.balance.isna()]

,user_id,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
2,120722,892.000,Рыбинск,0,30,0,NaN,1,1,1,107683.340,0
9,133130,906.000,Ярославль,0,67,0,NaN,1,0,1,238055.530,0
10,148929,927.000,Ростов,1,52,0,NaN,1,1,1,196820.070,0
11,172184,921.000,Ростов,1,41,0,NaN,1,1,1,217469.480,0
19,127034,922.000,Рыбинск,0,53,0,NaN,1,0,0,147094.820,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9976,208085,876.000,Ростов,1,38,0,NaN,1,0,0,171763.690,0
9984,125941,729.000,Ярославль,0,42,0,NaN,1,1,1,687538.700,0
9993,219924,884.000,Рыбинск,0,36,0,NaN,1,1,1,169844.880,0
9996,139170,894.000,Ярославль,1,46,0,NaN,1,1,0,196898.290,0


In [31]:
# Отсутсвие данных по балансу - скорее всего ошибка сбора/хранения данных, 
# в целях исследования заменим пропуски нулями:
df.balance = df.balance.fillna(0)

In [32]:
df.balance.isna().sum()

0

#### столбец products

In [33]:
df.products.value_counts()

2    5108
1    3340
3    1046
4     474
5      30
0       1
Name: products, dtype: int64

Отсутсвие продуктов - это скорее всего ошибка, удалим эту строку:

In [34]:
df = df.query('products > 0')

In [35]:
fig = px.histogram(df, x='products')
fig.show()

Больше всего клиентов с 2 банковскими продуктами

#### столбец credit_card

In [36]:
df.credit_card.value_counts()

1    6803
0    3195
Name: credit_card, dtype: int64

Кредитной картой пользуется почти 70% клиентов

#### столбец last_activity

In [37]:
df.last_activity.value_counts()

1    5235
0    4763
Name: last_activity, dtype: int64

Клиенты разделились примерно поровну на активных и неактивных пользователей

#### столбец est_salary

In [38]:
fig = px.histogram(df, x='est_salary')
fig.show()

In [39]:
fig = px.histogram(df, x='est_salary')
fig.update_layout(xaxis_range=[0, 200000])
fig.show()

Самая часто встречающаяся зарплата клиентов банка - около 100 тыс рублей 

####   Столбец churn

In [40]:
df.churn.value_counts()

0    8177
1    1821
Name: churn, dtype: int64

Видим, что данные несбалансированы: только 18% клиентов помечены, как ушедшие

Посмотрим корреляцию признаков:

In [41]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,user_id,score,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
user_id,1.000000,0.010039,0.030613,0.018040,0.004495,0.004429,-0.005948,0.005632,-0.031202,-0.000283,-0.012292
score,0.010039,1.000000,0.012748,-0.012590,0.064277,0.144463,-0.004444,-0.094597,-0.030879,0.163879,0.105868
gender,0.030613,0.012748,1.000000,-0.224822,-0.021463,0.032502,-0.023512,0.126147,-0.013093,0.082276,0.140999
age,0.018040,-0.012590,-0.224822,1.000000,0.032789,0.070970,0.032446,-0.131030,-0.003939,-0.042934,-0.056509
equity,0.004495,0.064277,-0.021463,0.032789,1.000000,0.252371,0.430285,-0.167425,-0.002090,-0.171781,0.270658
balance,0.004429,0.144463,0.032502,0.070970,0.252371,1.000000,0.155470,-0.083618,0.015892,0.159240,0.129848
products,-0.005948,-0.004444,-0.023512,0.032446,0.430285,0.155470,1.000000,-0.256778,0.039728,-0.119822,0.297725
credit_card,0.005632,-0.094597,0.126147,-0.131030,-0.167425,-0.083618,-0.256778,1.000000,-0.033534,0.035323,-0.131197
last_activity,-0.031202,-0.030879,-0.013093,-0.003939,-0.002090,0.015892,0.039728,-0.033534,1.000000,0.003123,0.169421
est_salary,-0.000283,0.163879,0.082276,-0.042934,-0.171781,0.159240,-0.119822,0.035323,0.003123,1.000000,0.001394


Удивительно, но однозначно линейно зависимых столбцов нет, отток нельзя объяснить каким-то одним признаком

<a id='data_preparation'></a>
### 3. Подготовка признаков

<div class="alert alert-danger">
<b>😔 Необходимо исправить:</b> Круто, а зачем здесь моделька и какую задачу она решает? Очень хочется подробностей, давь сюда описание происходящего
</div>

In [42]:
df.columns

Index(['user_id', 'score', 'city', 'gender', 'age', 'equity', 'balance',
       'products', 'credit_card', 'last_activity', 'est_salary', 'churn'],
      dtype='object')

In [43]:
# Удалим столбец user_id

In [44]:
df = df.drop(columns='user_id', axis=0)

In [45]:
df.columns

Index(['score', 'city', 'gender', 'age', 'equity', 'balance', 'products',
       'credit_card', 'last_activity', 'est_salary', 'churn'],
      dtype='object')

In [46]:
# score
fig = px.histogram(df, x='score', nbins=5)

fig.show()

In [47]:
def  set_score(x):
    if x <= 700:
        return  1
    elif x <= 800:
       return 2
    elif x <= 900:
      return 3
    elif x <= 1000:
       return 4
    else:
        return 5

In [48]:
a = 1000
set_score(a)

4

In [49]:
df.score = df.score.apply(lambda x : set_score(x))

In [50]:
df.head()

,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
0,3,Рыбинск,0,25,1,59214.820,2,0,1,75719.140,1
1,3,Рыбинск,0,37,5,850594.330,3,1,0,86621.770,0
2,3,Рыбинск,0,30,0,0.000,1,1,1,107683.340,0
3,3,Ярославль,0,51,5,1524746.260,2,0,1,174423.530,1
4,2,Ярославль,1,34,5,174.000,1,1,0,67353.160,1


In [51]:
# age
fig = px.histogram(df, x= 'age', nbins = 5)
fig.show()

In [52]:
def set_age(x):
    if x <=19:
        return 1
    elif x <=39:
        return 2
    elif x<=59:
        return 3
    elif x<=79:
        return 4
    else:
        return 5
    

In [53]:
df.age = df.age.apply(lambda x: set_age(x))

In [54]:
df.head()

,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
0,3,Рыбинск,0,2,1,59214.820,2,0,1,75719.140,1
1,3,Рыбинск,0,2,5,850594.330,3,1,0,86621.770,0
2,3,Рыбинск,0,2,0,0.000,1,1,1,107683.340,0
3,3,Ярославль,0,3,5,1524746.260,2,0,1,174423.530,1
4,2,Ярославль,1,2,5,174.000,1,1,0,67353.160,1


In [55]:
## balance
fig = px.histogram(df, x='balance')
fig.update_layout(xaxis_range = [0, 2000000])
fig.show()

In [56]:
def set_balance(x):
    if x <=50000:
        return 1
    elif x<=250000:
        return 2
    elif x<= 500000:
        return 3
    elif x<= 750000:
        return 4
    elif x <=1000000:
        return 5
    else: return 6

In [57]:
df.balance = df.balance.apply(lambda x: set_balance(x))

In [58]:
df.head()

,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
0,3,Рыбинск,0,2,1,2,2,0,1,75719.140,1
1,3,Рыбинск,0,2,5,5,3,1,0,86621.770,0
2,3,Рыбинск,0,2,0,1,1,1,1,107683.340,0
3,3,Ярославль,0,3,5,6,2,0,1,174423.530,1
4,2,Ярославль,1,2,5,1,1,1,0,67353.160,1


In [59]:
## est_salary
def set_salary(x):
    if x <= 50000:
        return 1
    elif x <= 100000:
        return 2
    elif x <= 150000:
        return 3
    elif x <= 200000:
        return 4
    else: return 5


In [60]:
df.est_salary = df.est_salary.apply(lambda x  : set_salary(x))

In [61]:
df.head()

,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
0,3,Рыбинск,0,2,1,2,2,0,1,2,1
1,3,Рыбинск,0,2,5,5,3,1,0,2,0
2,3,Рыбинск,0,2,0,1,1,1,1,3,0
3,3,Ярославль,0,3,5,6,2,0,1,4,1
4,2,Ярославль,1,2,5,1,1,1,0,2,1


<a id='modeling'></a>
### 4. Построение модели

In [62]:
#Разделим датасет на обучающую и тестовую выборки:
features = df.drop(columns='churn', axis=0)
target = df.churn
features_train, features_test, target_train, target_test \
    = train_test_split(features, target, train_size=0.75, random_state = 123, shuffle=target) 

In [63]:
(features_train.shape, target_train.shape)

((7498, 10), (7498,))

In [64]:
(features_test.shape, target_test.shape)

((2500, 10), (2500,))

In [65]:
# Создадим pipeline:
state = RandomState(123)

In [66]:
classifiers = [
  LogisticRegression(random_state=state),
  RandomForestClassifier(random_state=state),
  XGBClassifier(random_state=state),
  AdaBoostClassifier()
]

In [67]:
roc_auc_scorer = make_scorer(roc_auc_score, greater_is_better=True,
                             needs_threshold=True)

In [68]:
class_prarms = [
    { 'class__solver' : ['lbfgs', 'liblinear'],'class__penalty' :['l1','l2','none'], 'class__C': np.arange(0.5,1.5,0.25), 'class__fit_intercept' : [True, False] },
    {'class__max_features': ['auto','log2','sqrt', 'None'],'class__criterion' : ['gini', 'log_loss', 'entropy'], 'class__n_estimators' : range(10,1000,10), 'class__max_depth' : range(1,50), 'class__min_samples_split': range(1,10), 'class__min_samples_leaf': range(1,10)},
    {'class__eta': np.arange(0.01, 0.2, 0.01), 'class__max_depth': range(3,20), 'class__gamma' : np.arange(0,1,0.001),'class__learning_rate': np.arange(0.01, 1, 0.001), 'class__booster':['gbtree','gblinear'], 'class__n_estimators': range(10,1000,5)},
    {'class__n_estimators' : range(10,1000,10), 'class__learning_rate' : np.arange(0.1, 1, 0.01),'class__algorithm' : ['SAMME','SAMME.R'] }
]
cv = KFold(n_splits=10, shuffle=True, random_state=state)

In [69]:
cat_col = features.columns

In [70]:
t = [('cat', OneHotEncoder(), cat_col)]
col_transform = ColumnTransformer(transformers=t)

In [71]:
from scipy import rand


best_esimators = []
best_score = []
for i, classifier in enumerate(classifiers):
    steps = [
       ('prep',col_transform),
       ('smote', SMOTE(random_state=state)),
       ('selector', VarianceThreshold()),
       ('class', classifier)
    ]
    pipeline = Pipeline(steps)
    grid = RandomizedSearchCV(pipeline,class_prarms[i] , scoring=roc_auc_scorer, cv=cv, n_iter=10)
    grid.fit(features_train, target_train)
    print(classifier, grid.best_estimator_)
    best_esimators.append(grid.best_estimator_)
    best_score.append(grid.best_score_)
   

LogisticRegression(random_state=RandomState(MT19937) at 0x14C02C640) Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  Index(['score', 'city', 'gender', 'age', 'equity', 'balance', 'products',
       'credit_card', 'last_activity', 'est_salary'],
      dtype='object'))])),
                ('smote',
                 SMOTE(random_state=RandomState(MT19937) at 0x14C02E340)),
                ('selector', VarianceThreshold()),
                ('class',
                 LogisticRegression(C=0.5, fit_intercept=False, penalty='l1',
                                    random_state=RandomState(MT19937) at 0x14C02E440,
                                    solver='liblinear'))])
RandomForestClassifier(random_state=RandomState(MT19937) at 0x14C02C640) Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  Inde

In [92]:
best_score

[0.831257452456091, 0.8672375893700908, 0.864326539872575, 0.8281214958501744]

In [93]:
best_esimators[1]

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  Index(['score', 'city', 'gender', 'age', 'equity', 'balance', 'products',
       'credit_card', 'last_activity', 'est_salary'],
      dtype='object'))])),
                ('smote',
                 SMOTE(random_state=RandomState(MT19937) at 0x14C02F440)),
                ('selector', VarianceThreshold()),
                ('class',
                 RandomForestClassifier(max_depth=23, min_samples_leaf=4,
                                        min_samples_split=5, n_estimators=400,
                                        random_state=RandomState(MT19937) at 0x14C02F540))])

In [94]:
features_train_enc = pd.get_dummies(features_train, drop_first=True)

In [95]:
model = best_esimators[1][3]
model.fit(features_train_enc, target_train)
print(model.feature_importances_)

[0.08269056 0.061202   0.0780688  0.12941954 0.17201331 0.16370583
 0.05934595 0.11431784 0.08700944 0.02478539 0.02744133]


<a id='segmentation'></a>
### 5. Сегментирование

In [96]:
feat_importances = pd.Series(model.feature_importances_, index=features_train_enc.columns)

In [97]:
data = pd.Series(model.feature_importances_, index=features_train_enc.columns).to_frame()

In [98]:
fig = px.bar(data.sort_values(by=0), orientation='h')
fig.update_layout(
    title="Влияние параметров на предсказание",
    xaxis_title="параметр",
    yaxis_title="важность")
fig.update_layout(showlegend=False)
fig.show()

Видим, что наибольшим влиянием на вероятность ухода клиента обладают:  
* balance  
* products  
* equity  
* last activity 

In [99]:
df.groupby('balance', as_index=False).agg({'churn':'mean'})\
    .sort_values(by='churn', ascending=False).reset_index(drop=True)

,balance,churn
0,6,0.385
1,5,0.264
2,4,0.192
3,3,0.173
4,2,0.154
5,1,0.018


В группе клиентов с максимальным балансом на счету (от 1 млн рублей) самый большой отток - почти 40%

In [100]:
df.groupby('products', as_index=False).\
    agg({'churn':'mean'}).sort_values(by='churn', ascending=False).reset_index(drop=True)

,products,churn
0,4,0.633
1,5,0.300
2,3,0.285
3,2,0.192
4,1,0.070


В группе клиентов с 4 продуктами максимальный отток - 63%

In [101]:
df.groupby('equity', as_index=False).\
    agg({'churn':'mean'}).sort_values(by='churn', ascending=False).reset_index(drop=True)

,equity,churn
0,9,0.538
1,7,0.463
2,6,0.360
3,8,0.353
4,5,0.301
5,4,0.251
6,3,0.209
7,2,0.158
8,1,0.120
9,0,0.035


В группе клиентов с более чем 5 объектами недвижимости самый большой отток

In [102]:
df.groupby('last_activity', as_index=False).\
    agg({'churn':'mean'}).sort_values(by='churn', ascending=False).reset_index(drop=True)

,last_activity,churn
0,1,0.245
1,0,0.114


In [113]:
#
balance_products = df.pivot_table(
    index='balance', columns='products', values='churn', aggfunc='mean')
fig = px.imshow(balance_products, text_auto=True, aspect="auto" , title = 'Средний отток в зависимости от баланса и количества продуктов')
fig.show()

In [118]:
#
balance_products = df.pivot_table(
    index='balance', columns='equity', values='churn', aggfunc='mean')
fig = px.imshow(balance_products, text_auto=True, aspect="auto" , title = 'Средний отток в зависимости от баланса и недвижимости')
fig.show()

In [117]:
first_segment = df.query('balance >=5 and products >=3 and equity >=5 and last_activity==1')
first_segment.head()

,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
72,3,Ярославль,1,2,5,5,3,0,1,4,1
76,4,Ярославль,0,2,5,6,4,0,1,2,0
148,3,Ростов,0,3,5,6,4,0,1,2,1
183,3,Ярославль,0,2,5,5,4,0,1,3,0
216,3,Рыбинск,0,3,5,5,4,0,1,2,0


Те клиенты, которые недавно были активны вдвое чаще уходят

Сформируем кластеры клиентов:  
**Клиенты склонные уходить**:  
- Имеют на балансе более `750 000` руб  
- Владеют `3-5` продуктами  
- Владеют  `более, чем 5` объектами недвижимости  
- Были `активны` в предшествующий период  



**Лояльные клиенты**  
- Имеют на балансе менее `50 000` руб  
- Владеют `1-2` продуктами  
- Владеют  `менее, чем 4` объектами недвижимости  
- Были `неактивны` в предшествующий период  

In [103]:
df.query('balance >=5 and products >=3 and equity >=5 and last_activity==1')['churn']\
    .mean().round(decimals=3)

0.684

In [104]:
df.query('balance <=1 and products <=2 and equity <=3 and last_activity==0')['churn']\
    .mean().round(decimals=3)

0.016

В первом сегменте уходит 68% клиентов, во втором - 16

<div class="alert alert-danger">
<b>😔 Необходимо исправить:</b> Какова численность этих сегментов? Предлагаю выделить еще несколько сегментов и посмотреть отток
</div>

<a id='statistics'></a>
### 6. Проверка гипотез

Сформулируем статистические гипотезы: Гипотеза о равенстве дохода 
HO: доход в 1 и 2 сегменте не отличается  
H1: доход в 1 и 2 сегменте отличаются

In [85]:
first_segment = df.query('balance >=5 and products >=3 and equity >=5 and last_activity==1')
second_segment = df.query('balance <=1 and products <=2 and equity <=3 and last_activity==0')

In [107]:
first_segment.head()

,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
72,3,Ярославль,1,2,5,5,3,0,1,4,1
76,4,Ярославль,0,2,5,6,4,0,1,2,0
148,3,Ростов,0,3,5,6,4,0,1,2,1
183,3,Ярославль,0,2,5,5,4,0,1,3,0
216,3,Рыбинск,0,3,5,5,4,0,1,2,0


In [105]:
first_segment.shape[0]

187

In [106]:
second_segment.shape[0]

1081

In [86]:
alpha = 0.05
results = st.ttest_ind(first_segment['est_salary'], second_segment['est_salary'])
if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу, pvalue =', results.pvalue)
else:
    print('Не можем отвергнуть нулевую гипотезу')

Не можем отвергнуть нулевую гипотезу


Нет данных утвреждать, что доход в сегментах отличается

Гипотеза о различном возрасте клиентов в сегментах:
HO: возраст клиентов в сегментах отличается  
H1: возраст клиентов в сегментах не отличается

In [87]:
alpha = 0.05
results = st.ttest_ind(first_segment['age'], second_segment['age'])
if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу, pvalue =', results.pvalue)
else:
    print('Не можем отвергнуть нулевую гипотезу')

Отвергаем нулевую гипотезу, pvalue = 0.0004919774194565032


Получили данные, что возраст в сегментах отличается, посмотрим внимательнее:  


In [88]:
first_segment.age.median()

3.0

In [89]:
fig = px.histogram(first_segment, x='age', 
title='Распределение клиентов в "проблемном" сегменте по возрасту')
fig.show()

In [90]:
second_segment.age.median()

2.0

In [91]:
fig = px.histogram(second_segment, x='age',
 title = 'Распределение клиентов в "лояльном" сегменте по возрасту')
fig.show()

Как видим,  возраст клиентов в Сегменте 1 - 60-80 лет, в Сегменте 2 - 40-60 лет

<div class="alert alert-danger">
<b>😔 Необходимо исправить:</b> Визуализация тоже не отображается(((
</div>

<a id='conclusion'></a>
### 7. Выводы и предложения


1. Провели анализ датасета для анализа оттока клиентов банка  
2. Провели предобработку данных, разделили данные на группы для аналиа  
3. Нашли параметры, оказывающие наибольшее влияние на отток, ими оказались:  
- balance,  
- products,  
- equity,  
- last_activity  
4. Построили сегменты: "уходящих" - сегмент 1 и "лояльных" - сегмент 2 клиентов  
  
  
**Проблемный сегмент - Сегмент 1** :
- Имеют на балансе более `750 000` руб  
- Владеют `3-5` продуктами  
- Владеют  `более, чем 5` объектами недвижимости  
- Были `активны` в предшествующий период  
  
  
****Лояльный сегмент - Сегмент 2** :  
- Имеют на балансе менее `50 000` руб  
- Владеют `1-2` продуктами  
- Владеют  `менее, чем 4` объектами недвижимости  
- Были `неактивны` в предшествующий период  
5. Сформулировали и проверили статистические гипотезы о равенстве дохода в сегментах и о равнестве возраста в сегментах. Получили, что нет данных о том, что доход в сегментах отличается.  
Возраст же клиентов в сегментах статистически различается, возраст клиентов в проблемном сегменте выше - 60-80 лет, а в лояльном сегменте - 40-60 лет  
 


### Предложения  
Получили несколько удивительные данные: наиболее склонны уходить клиенты с большим балансом, владеющие разнообразной недвижмостью, активные, обдадающие большим количеством продуктов.  
Очевидно, что условия обслуживания "премиальных" клиентов в банке не соответствуют их ожиданиям.  Необходимо скорректировать работу с  премиальными клиентами, исследовать рынкок конкурентов, сконцентрировать работу на выработке предложений и условий для удержания проблемного сегмента

<div class="alert alert-danger">
<b>😔 Необходимо исправить:</b> Хорошая работа, но нужно добавить больше комментариев и объяснений что и зачем происходит, поправить визуализацию 
</div>

[Презентация](https://github.com/kbzunder/Yandex_Practicum_Projects/blob/main/final/plots/Banks_final.pdf)